In [49]:
%pip install tensorflow tf-keras -U
%pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [24]:
import os
import re
import shutil
import string
import tensorflow as tf
import pandas as pd
import numpy as np

tf.random.set_seed(42)
print("TensorFlow version:", tf.__version__)

VOCAB_SIZE = 512
EMBEDDING_DIM = 128
MAX_LENGTH = 32
TRAINING_SPLIT = 0.8

TensorFlow version: 2.18.0


In [25]:
df = pd.read_csv("titles-labelled-nathan.csv")
# Shuffle
df = df.sample(frac=1)

train_size = int(len(df) * TRAINING_SPLIT)

titles = df['title'].values
labels = df['is_transaction'].values.astype(int)

# Split them
train_titles = titles[:train_size]
train_labels = labels[:train_size]
test_titles = titles[train_size:]
test_labels = labels[train_size:]

# Create the datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_titles, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_titles, test_labels))

In [26]:
print(f"There are {train_dataset.cardinality()} sentence-label pairs for training.\n")
print(f"There are {test_dataset.cardinality()} sentence-label pairs for validation.\n")

There are 8000 sentence-label pairs for training.

There are 2000 sentence-label pairs for validation.



In [32]:
vectorizer = tf.keras.layers.TextVectorization(
	max_tokens=VOCAB_SIZE,
    output_sequence_length=MAX_LENGTH,
    pad_to_max_tokens=True,
)

vectorizer.adapt(train_dataset.map(lambda title, is_transaction: title))
vocab_size = vectorizer.vocabulary_size()

# Save vectorizer
import pickle
pickle.dump({'config': vectorizer.get_config(),
             'vocabulary': vectorizer.get_vocabulary()}
            , open("tv_layer.pkl", "wb"))

print("Vocabulary size: {}".format(vocab_size))

Vocabulary size: 512


In [33]:
print(titles[0])
vectorizer(titles[0])

Checkout Pesanan dengan GoPay Tabungan by Jago Berhasil tanggal 19 September 2024 from:tokopedia.com


<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([ 21,   4,  19,  14,  25,  26,  16,  17,  20, 380,  38,  15,   7,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0], dtype=int64)>

In [34]:
def preprocess_dataset(dataset, text_vectorizer):
    """Apply the preprocessing to a dataset

    Args:
        dataset (tf.data.Dataset): dataset to preprocess
        text_vectorizer (tf.keras.layers.TextVectorization ): text vectorizer

    Returns:
        tf.data.Dataset: transformed dataset
    """
    # Convert the Dataset sentences to sequences, and encode the text labels
    dataset = dataset.map(lambda text, label: (text_vectorizer(text), label), num_parallel_calls=32).batch(32)
    
    return dataset

In [35]:
train_proc_dataset = preprocess_dataset(train_dataset, vectorizer)
test_proc_dataset = preprocess_dataset(test_dataset, vectorizer)

print(f"Number of batches in the train dataset: {train_proc_dataset.cardinality()}")
print(f"Number of batches in the validation dataset: {test_proc_dataset.cardinality()}")

Number of batches in the train dataset: 250
Number of batches in the validation dataset: 63


In [36]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(MAX_LENGTH,)),
    tf.keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_proc_dataset, epochs=10, validation_data=test_proc_dataset)
test_loss, test_acc = model.evaluate(test_proc_dataset)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7399 - loss: 0.5229 - val_accuracy: 0.9565 - val_loss: 0.1269
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9513 - loss: 0.1602 - val_accuracy: 0.9585 - val_loss: 0.1296
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9549 - loss: 0.1472 - val_accuracy: 0.9600 - val_loss: 0.1137
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9562 - loss: 0.1274 - val_accuracy: 0.9600 - val_loss: 0.1088
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9559 - loss: 0.1266 - val_accuracy: 0.9630 - val_loss: 0.1177
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9566 - loss: 0.1244 - val_accuracy: 0.9645 - val_loss: 0.1084
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9593 - loss: 0.1150 - val_accuracy: 0.9615 - val_loss: 0.1040
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9584 - loss: 0.1148 - val_accuracy: 0.

In [ ]:
# Save model
model.save('email_titles_nlp.keras')

: 